In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#Importation des librairies
import warnings
warnings.filterwarnings("ignore")

import os
import re
import glob
import pandas as pd
import missingno as msno
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

In [3]:
#On modifie les options pour rendre l'affichage des float plus lisible
pd.set_option('float_format', '{:f}'.format)

In [ ]:
#On définit les colonnes du dataset qui nous serons utiles 
fields = ['code', 'product_name','generic_name','brands','categories','origins','labels','cities','purchase_places','stores','countries','ingredients_text','allergens','additives_n','nutriscore_score','nutriscore_grade','nova_group','pnns_groups_1','pnns_groups_2','ecoscore_score_fr','ecoscore_grade_fr','main_category','image_url','energy-kj_100g',
'energy-kcal_100g','salt_100g','sodium_100g','fat_100g','saturated-fat_100g','sugars_100g','fiber_100g','proteins_100g','carbohydrates_100g','alcohol_100g','fruits-vegetables-nuts_100g']

df = pd.read_csv("en.openfoodfacts.org.products.csv", sep='\t', usecols=fields)

Les colonnes ont été choisies en fonction de nos besoins pour l'application et suite à diverses recherches et tests statistiques sur le dataset

Dans un premier temps nous allons vérifier le nombre de produits dans le dataset puis commencer à éliminer toutes les entrées ne correspondant pas à nos besoins pour l'application. Ici nous retirerons alors les données dont le nom est absent ainsi que les données de produits non français par soucis de lisibilité et de disponibilité en magasin.

In [ ]:
print("Données de base : ")
print(df['code'].describe())

In [ ]:
df = df[df['product_name'].str.strip().astype(bool)]
print("Après filtre non vide : ")
print(df['code'].describe())

In [ ]:
df = df[df['countries'] == 'France']
print("Après filtre France : ")
print(df['code'].describe())

In [ ]:
df['categories'].replace('', np.nan, inplace=True)
df.dropna(subset=['categories'], inplace=True)
print(df['code'].describe())

Nous supprimons toutes les catégories vides car il nous est impossible de proposer un produit similaire au produit entré sans connaitre sa catégorie

Nous allons maintenant utiliser la librairie Missingno afin de voir la quantité de données vides dans notre jeu de données

In [ ]:
msno.matrix(df)

In [ ]:
for var in ["salt_100g","sodium_100g","fat_100g","saturated-fat_100g","sugars_100g","carbohydrates_100g","fiber_100g","proteins_100g"]:
    sns.boxplot(x=var, data=df)
    plt.title(var)
    plt.show()

On remarque assez vite ici qu'il y a des valeurs dépassant le seuil de 100grammes. Il s'agit probablement de valeurs entrées en miligrammes au lieu de grammes mais nous allons les supprimer car cela demanderait une vérification manuelle pour chaque produit.

Nous allons donc procéder à un nettoyage de toutes les valeurs en dehors des bornes 0 à 100, sauf pour le sodium qui sera entre les bornes 0 à 40 car il ne peut pas excéder cette valeur, nous verrons pourquoi par la suite.

In [ ]:
df = df[df['salt_100g']>=0]
df = df[df['salt_100g']<=100.0]

df = df[df['sodium_100g']>=0]
df = df[df['sodium_100g']<=40]

df = df[df['fat_100g']>=0]
df = df[df['fat_100g']<=100.0]

df = df[df['saturated-fat_100g']>=0]
df = df[df['saturated-fat_100g']<=100.0]

df = df[df['sugars_100g']>=0]
df = df[df['sugars_100g']<=100.0]

df = df[df['fiber_100g']>=0]
df = df[df['fiber_100g']<=100.0]

df = df[df['carbohydrates_100g']>=0]
df = df[df['carbohydrates_100g']<=100.0]

df = df[df['proteins_100g']>=0]
df = df[df['proteins_100g']<=100.0]

Nous estimerons également que toutes les valeurs nulles pour la colonne alcohol_100g sont des produits ne contenant pas d'alcool, nous les mettrons alors d'office à 0 grammes pour la suite de nos calculs

In [ ]:
df['alcohol_100g'] = df['alcohol_100g'].fillna(0)

In [ ]:
for var in ["salt_100g","sodium_100g","fat_100g","saturated-fat_100g","sugars_100g","carbohydrates_100g","fiber_100g","proteins_100g"]:
    sns.boxplot(x=var, data=df)
    plt.title(var)
    plt.show()

Nous avons maintenant des boxplot correspondant bien mieux à ce que l'on pouvait s'attendre. On remarque également que le boxplot de sel et celui de sodium ont l'air similaires à leur échelle. Il est donc intéressant de voir s'il y a des corrélations entre les différentes variables que nous avons.

Pour cela nous allons réaliser une heatmap relevant les corrélations entre chacune de ces variables sur une échelle de 1 à 10:

In [ ]:
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

Nous pouvons distinguer que le sel et le sodium sont très corrélés ainsi que la graisse et la graisse saturée, nous allons donc tester ces valeurs avec un scatterplot pour déterminer s'il y a une relation entre ces deux variables

In [ ]:
dfSample = df.sample(50000) # Nous prenons un échantillon de 50 000 entités pour accélérer le processus
xdataSample, ydataSample = dfSample["salt_100g"], dfSample["sodium_100g"]

sns.scatterplot(data=dfSample, x="salt_100g", y="sodium_100g") 
plt.show()

Nous pouvons effectivement voir une belle relation entre ces deux variables grâce à cette diagonale sur le graphique.

Après quelques recherches nous trouvons rapidement qu'il existe bel et bien une relation d'un coefficient de 2,54 entre ces deux variables (1gramme de sodium équivaut à 2,54 grammes de sel), ce qui nous permet de savoir qu'il n'y aura jamais plus de 40grammes de sodium dans une portion de 100grammes d'aliment

In [ ]:
dfSample = df.sample(50000) # Nous prenons un échantillon de 50 000 entités pour accélérer le processus
xdataSample, ydataSample = dfSample["fat_100g"], dfSample["saturated-fat_100g"]

sns.scatterplot(data=dfSample, x="fat_100g", y="saturated-fat_100g") 
plt.show()

Pour les graisses et les graisses saturées nous pouvons également relever une corrélation mais il n'y a pas de coefficient visible entre ces deux variables. On peut néamoins remarquer qu'il parait impossible que les graisses saturées dépasse le taux de graisses, mis à part quelques cas qui sont potentiellement des erreurs.

Maintenant que nous connaissons la relation entre le sel et le sodium nous allons pouvoir procéder à une vérification pour chaque produit afin d'être sur que les données soient qualitatives

In [ ]:
print("Avant vérification :",df["code"].count())
df = df.loc[abs(df['salt_100g']-df['sodium_100g']*2.54) < df['salt_100g']*0.05]
print("Après vérification :",df["code"].count())



En ayant mis une tolérance de 5% de différence entre les valeurs données et les valeurs calculées nous avons quand même exclus une bonne partie des données qui sont potentiellement erronées

En faisant nos recherches sur la relation entre le sel et le sodium nous avons découvert que la graisse comprends les graisses saturées et que les glucides comprennent le sucre et les fibres.
Nous pouvons donc vérifier que ces valeurs ne dépassent pas le taux de graisse et le taux de glucides liés.

In [ ]:
print("Avant vérification :",df["code"].count())
df = df.loc[df['fat_100g'] >= df['saturated-fat_100g']]
print("Après vérification :",df["code"].count())


In [ ]:
print("Avant vérification :",df["code"].count())
df = df.loc[df['carbohydrates_100g'] >= df['sugars_100g']+df["fiber_100g"]]
print("Après vérification :",df["code"].count())

Nous pouvons donc déduire avec ces dernières informations que toutes nos variables sont regroupées dans les 4 plus importantes, à savoir : graisses, glucides, protéines, sel.
Il est donc important de vérifier que la somme de ces 4 variables ne dépasse pas 100grammes.

In [ ]:
print("Avant vérification :",df["code"].count())
df = df.loc[df['fat_100g'] + df['carbohydrates_100g']+df["salt_100g"]+df["proteins_100g"]<=100]
print("Après vérification :",df["code"].count())

Nous allons maintenant réutiliser Missingno afin de voir si nos données sont toutes remplies ou non

In [ ]:
msno.matrix(df)

On peut remarquer ici que les colonnes comprenant les valeurs nutritives de l'aliment sont complètement remplies, ce qui va nous permettre de recalculer les valeurs manquantes de la valeur énérgétique en kcal et kj ainsi que le nutriscore.

En effet, la valeur énergétique en kcal d'un produit est égale à : glucides(gr) \* 4 + protéines(gr) \* 4 + graisses(gr) \* 9 + Alcool(gr) \* 7

On sait également que la valeur en kj d'un produit est égale à : kcal * 4.184
Maintenant que l'on sait que notre valeur en kcal est correcte, on peut remplacer toutes les valeurs en kj sans distinction

In [ ]:
df['energy-kcal_100g']=df['fat_100g']*9+df['carbohydrates_100g']*4+df['proteins_100g']*4+df['alcohol_100g']*7
df['energy-kj_100g']=df['energy-kcal_100g']*4.184

In [ ]:
msno.matrix(df)

In [ ]:
sns.countplot(x="nutriscore_score", data=df)
plt.title("nutriscore_score")
plt.xticks(rotation=90)
plt.show()


sns.countplot(x="nutriscore_grade", data=df, order =["a","b","c","d","e"])
plt.title("nutriscore_grade")
plt.show()

In [ ]:
#Calcul du nutriscore

for index, row in df.iterrows():
    point_n = 0
    sugar = row["sugars_100g"]
    energy = row["energy-kj_100g"]
    if("Boissons" in row["categories"]):
        point_n += 10 if sugar > 45 else (9 if sugar > 40 else (8 if sugar > 36 else(7 if sugar > 31 else(6 if sugar > 27 else(5 if sugar > 22.5 else(4 if sugar > 18 else(3 if sugar > 13.5 else(2 if sugar > 9 else(1 if sugar > 4.5 else 0)))))))))
        point_n += 10 if energy > 270 else (9 if energy > 240 else (8 if energy > 210 else(7 if energy > 180 else(6 if energy > 150 else(5 if energy > 120 else(4 if energy > 90 else(3 if energy > 60 else(2 if energy > 30 else(1 if energy > 0 else 0)))))))))
    else:
        point_n += 10 if sugar > 13.5 else (9 if sugar > 12 else (8 if sugar > 10.5 else(7 if sugar > 9 else(6 if sugar > 7.5 else(5 if sugar > 6 else(4 if sugar > 4.5 else(3 if sugar > 3 else(2 if sugar > 1.5 else(1 if sugar > 0 else 0)))))))))
        point_n += 10 if energy > 3350 else (9 if energy > 3015 else (8 if energy > 2680 else(7 if energy > 2345 else(6 if energy > 2010 else(5 if energy > 1675 else(4 if energy > 1340 else(3 if energy > 1005 else(2 if energy > 670 else(1 if energy > 335 else 0)))))))))
    saturated_fat = row["saturated-fat_100g"]
    if("Matières grasses" in row["categories"]):
        point_n += 10 if saturated_fat >= 64 else (9 if saturated_fat >= 58 else (8 if saturated_fat >= 52 else(7 if saturated_fat >= 46 else(6 if saturated_fat >= 40 else(5 if saturated_fat >= 34 else(4 if saturated_fat >= 28 else(3 if saturated_fat >= 22 else(2 if saturated_fat >= 16 else(1 if saturated_fat >= 10 else 0)))))))))
    else:
        point_n += 10 if saturated_fat > 10 else (9 if saturated_fat > 9 else (8 if saturated_fat > 8 else(7 if saturated_fat > 7 else(6 if saturated_fat > 6 else(5 if saturated_fat > 5 else(4 if saturated_fat > 4 else(3 if saturated_fat > 3 else(2 if saturated_fat > 2 else(1 if saturated_fat > 1 else 0)))))))))
    sodium = row["sodium_100g"]
    point_n += 10 if sodium > 900 else (9 if sodium > 810 else (8 if sodium > 720 else(7 if sodium > 630 else(6 if sodium > 540 else(5 if sodium > 450 else(4 if sodium > 360 else(3 if sodium > 270 else(2 if sodium > 180 else(1 if sodium > 90 else 0)))))))))
    
    point_p = 0
    point_fruits = 0
    point_fibers = 0
    point_proteins = 0
    fruits = row["fruits-vegetables-nuts_100g"]    
    fibers = row["fiber_100g"]
    proteins = row["proteins_100g"]
    if("Boissons" in row["categories"]):
        point_fruits += 10 if fruits >80 else (4 if fruits >60 else (2 if fruits >40 else 0))
    else:
        point_fruits += 5 if fruits > 80 else (2 if fruits > 60 else (1 if fruits > 40 else 0))
    point_fibers += 5 if fibers > 3.5 else (4 if fibers > 2.8 else (3 if fibers > 2.1 else (2 if fibers > 1.4 else (1 if fibers > 0.7 else 0))))
    point_proteins += 5 if proteins > 8 else (4 if proteins > 6.4 else (3 if proteins > 4.8 else (2 if proteins > 3.2 else (1 if proteins > 1.6 else 0))))
    
    point_p = point_fruits + point_fibers + point_proteins
    
    #Choix de la méthode de calcul du nutriscore:
    if("Matières grasses" in row["categories"] or point_n < 11):
        score = point_n - point_p
    elif(point_fruits <5):
        score = point_n - (point_fibers + point_fruits)
    else :
        score = point_n - point_p
    
    #Réattribution du score 
    row["nutriscore_score"] = score

    #Réattribution du rang
    if(score <0 and "Boissons" not in ["categories"]):
        row["nutriscore_grade"] = "a"
    elif(0 <= score <= 2 and "Boissons" not in ["categories"]):
        row["nutriscore_grade"] = "b"
    elif(3 <= score <= 10 and "Boissons" not in ["categories"] or (2 <= score <= 5 and "Boissons" in ["categories"])):
        row["nutriscore_grade"] = "c"
    elif(11 <= score <= 18 and "Boissons" not in ["categories"] or (6 <= score <= 9 and "Boissons" in ["categories"])):
        row["nutriscore_grade"] = "d"
    elif(19 <= score and "Boissons" not in ["categories"] or (10 <= score and "Boissons" in ["categories"])):
        row["nutriscore_grade"] = "e"
           

Le nutriscore est recalculé entièrement pour tous les produits afin de vérifier avec nos données nettoyées si les scores correspondent toujours. Ensuite on attribue le rang de nutriscore en fonction du score obtenu avec nos variables.

On a choisi de ne pas modifier le rang pour les boissons en dessous du score de 2 car il nous était impossible de déterminer si la boisson était de l'eau ou non, le rang reste donc le même qu'avant le calcul.

In [ ]:
sns.countplot(x="nutriscore_score", data=df)
plt.title("nutriscore_score")
plt.xticks(rotation=90)
plt.show()


sns.countplot(x="nutriscore_grade", data=df, order =["a","b","c","d","e"])
plt.title("nutriscore_grade")
plt.show()

On peut voir ici que les deux graphiques correspondent complètement aux deux autres graphiques réalisés avec les données avant calcul. On peut donc déduire que notre formule est correcte.

In [ ]:
print(df["categories"].describe())